In [0]:
!pip install docling
!pip install -qU pip docling transformers

In [0]:
from pathlib import Path
from pyspark.dbutils import DBUtils
import re, os

def get_repo_root() -> Path:
    dbutils = DBUtils(spark)
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

    # Extrahiere /Workspace/Repos/<user>/<repo>
    match = re.match(r"^(\/Workspace\/Repos\/[^\/]+\/[^\/]+)", notebook_path)
    if match:
        return Path(match.group(1))
    else:
        return Path(os.getcwd())

repo_root = get_repo_root()
print("📁 Repo root:", repo_root)

# Eine Ebene höher (wie "cd ..")
parent_path = repo_root.parent

# Zwei Ebenen höher
two_up = repo_root.parent.parent

print("⬆️ Eine Ebene höher:", parent_path)
print("⬆️ root:", parent_path)

doc_root = parent_path / "documents"
out_dir = parent_path / "out"
product_name = "Nano 33 BLE"
product_category = "Nano Family"
file_name  = "Nano_33_BLE_datasheet.pdf"

pdf_path = doc_root / product_category / product_name / "Nano_33_BLE_datasheet.pdf"
print("⬆️ pdf pfad:",pdf_path)

out_path_docling = out_dir / product_category / product_name / "docling_chunks.jsonl"
print("⬆️ oout_path_docling:",out_path_docling)

out_path_langchain = out_dir / product_category / product_name / "langchain_chunks.jsonl"
print("⬆️ out_path_langchain:",out_path_langchain)

code_path_langchain = parent_path / "langchain_chunking.py"
print("⬆️code_path_langchain:",code_path_langchain)

code_path_docling= parent_path / "docling_chunking.py"
print("⬆️code_path_langchain:",code_path_docling)




In [0]:
from pathlib import Path
import re
from pyspark.dbutils import DBUtils
import os

def get_repo_root() -> Path:
    dbutils = DBUtils(spark)
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

    match = re.match(r"^(\/Workspace\/Users\/[^\/]+\/[^\/]+)", notebook_path)
    if match:
        return Path(match.group(1))
    else:
        return Path(os.getcwd())

repo_root = get_repo_root()
print("📁 Repo root:", repo_root)

parent_path = repo_root.parent
doc_root = parent_path / "documents"
out_dir = parent_path / "out"

# ✅ Liste der zu verarbeitenden Unterordner
allowed_folders = [
    "Arduino Cloud",
    "Arduino CLI",
]

def iterate_product_docs(doc_root: Path, out_dir: Path):
    """Durchläuft nur die erlaubten Unterordner von documents/"""
    for folder in allowed_folders:
        folder_path = doc_root / folder
        if not folder_path.exists():
            print(f"⚠️  Ordner nicht gefunden: {folder_path}")
            continue

        # jetzt PDFs innerhalb dieses Ordners suchen
        for pdf_path in folder_path.rglob("*.pdf"):
            if len(pdf_path.parts) >= 3:
                print(f"📂 {pdf_path.parent.parent.name} / {pdf_path.parent.name}")
                #process_pdf(pdf_path, out_dir)

# 🚀 Aufruf
iterate_product_docs(doc_root, out_dir)



In [0]:
pdf_path = "/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/documents/Arduino Cloud/Cloud Editor/Cloud Editor.pdf"

In [0]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
pdf_options = PdfPipelineOptions()
pdf_options.do_ocr = False                    # No OCR - pure text extraction only   # noqa: E501
pdf_options.generate_page_images = False      # No page images  # noqa: E501
pdf_options.generate_picture_images = False   # Ignore pictures completely  # noqa: E501
pdf_options.generate_table_images = False     # Keep tables as text/markdown, not images  # noqa: E501

        # Configure format options
format_options = {
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pdf_options
            )
        }

# Initialize document converter
converter_2 = DocumentConverter(
            format_options=format_options
        )
result_2 = converter_2.convert(pdf_path)
doc_2 = result_2.document

In [0]:
from pathlib import Path

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    TesseractCliOcrOptions,
)
from docling.document_converter import DocumentConverter, PdfFormatOption

ocr_options = TesseractCliOcrOptions(lang=["auto"])

pipeline_options = PdfPipelineOptions(
        do_ocr=True, force_full_page_ocr=True, ocr_options=ocr_options
    )

converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options,
            )
        }
    )

doc = converter.convert(pdf_path).document
md = doc.export_to_markdown()
print(md)

In [0]:
from pathlib import Path

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    TesseractCliOcrOptions,
)
from docling.document_converter import DocumentConverter, PdfFormatOption


ocr_options = TesseractCliOcrOptions(lang=["auto"])

pipeline_options = PdfPipelineOptions(
        do_ocr=True, force_full_page_ocr=True, ocr_options=ocr_options
    )

converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options,
            )
        }
    )

doc = converter.convert(pdf_path).document
md = doc.export_to_markdown()
print(md)

In [0]:
%pip install pymupdf

In [0]:
import fitz  # PyMuPDF
from PIL import Image
import pytesseract

def ocr_pdf_with_tesseract(pdf_path: str, dpi: int = 300, lang: str = "eng+deu") -> str:
    doc = fitz.open(pdf_path)
    out = []
    for page in doc:
        # 1) Normale Textschicht versuchen
        txt = page.get_text("text").strip()
        if txt:
            out.append(txt)
            continue
        # 2) Sonst OCR auf gerenderter Seite
        pix = page.get_pixmap(dpi=dpi)
        mode = "RGB" if pix.n < 4 else "RGBA"
        img = Image.frombytes(mode, [pix.width, pix.height], pix.samples)
        out.append(pytesseract.image_to_string(img, lang=lang))
    doc.close()
    return "\n\n".join(out).strip()

raw_text = ocr_pdf_with_tesseract(pdf_path, dpi=300, lang="eng+deu")
print(raw_text[:1200])


In [0]:
import os, re, json
from pathlib import Path

# ========= 1) Hilfsfunktionen =========
def is_heading(line: str) -> bool:
    s = line.strip()
    if not s:
        return False
    # kurz + kaum Satzzeichen
    if len(s) <= 80 and sum(ch in ".:;|/[]()" for ch in s) <= 1:
        # ALL CAPS oder Title Case oder endet nicht mit Punkt
        if (s.isupper() and any(c.isalpha() for c in s)) or \
           (s.istitle() and " " in s) or \
           not s.endswith("."):
            # wenig Ziffern
            digits = sum(ch.isdigit() for ch in s)
            if digits <= max(2, len(s)//10):
                return True
    return False

def split_into_sections(cleaned_text: str):
    sections = []
    cur_title = ""
    cur_buf = []
    for line in cleaned_text.splitlines():
        if is_heading(line):
            # flush
            if cur_buf:
                sections.append((cur_title, "\n".join(cur_buf).strip()))
                cur_buf = []
            cur_title = line.strip()
        else:
            cur_buf.append(line)
    if cur_buf:
        sections.append((cur_title, "\n".join(cur_buf).strip()))
    # Notfall: wenn keine Headings erkannt → alles in eine Section
    if not sections:
        sections = [("", cleaned_text)]
    return sections

def approx_token_count(text: str) -> int:
    # sehr grobe Approximation: ~4 Zeichen pro Token
    return max(1, int(len(text) / 4))

def semantic_density_heuristic(text: str) -> float:
    # einfache, stabile Heuristik: Anteil einzigartiger Wörter * Bonus für Satzlänge
    words = re.findall(r"[A-Za-zÀ-ÿ0-9]+", text.lower())
    if not words:
        return 0.0
    uniq = len(set(words))
    ratio = uniq / len(words)                      # 0..1
    # längere Sätze deuten oft auf inhaltliche Dichte
    sentences = re.split(r"[.!?]\s+", text.strip())
    avg_len = sum(len(s.split()) for s in sentences if s) / max(1, len(sentences))
    bonus = min(1.0, avg_len / 25.0)               # capped
    score = 0.6 * ratio + 0.4 * bonus
    return round(min(1.0, max(0.0, score)), 3)

def chunk_section(text: str, target_tokens=800, overlap_tokens=120):
    # wir chunk’en auf Token-Approximation; schneiden an Wortgrenzen
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        # Zielspanne in Wörtern schätzen (1 Token ~ 0.75 Wort bei dieser Approx)
        target_words = int(target_tokens * 0.75)
        end = min(len(words), start + target_words)
        chunk = " ".join(words[start:end]).strip()
        if chunk:
            chunks.append(chunk)
        if end == len(words):
            break
        # Overlap
        overlap_words = int(overlap_tokens * 0.75)
        start = max(start + target_words - overlap_words, start + 1)
    return chunks

# ========= 2) Hauptroutine: OCR-Text → Clean → Sections → Chunks → JSONL =========
def chunk_ocr_text_to_jsonl(
    raw_text: str,
    category: str,
    product: str,
    out_dir: str,
    base_id: str = "ocr",
    target_tokens: int = 800,
    overlap_tokens: int = 120
):
    # 2.1 Cleaning – nimm hier DEINE erweiterte Funktion
    def clean_html_extracted_pdf_text(t: str) -> str:
        # >>>> ERSETZE bei Bedarf durch deine Version mit Ligaturen etc. <<<<
        t = re.sub(r"(\w)-\n(\w)", r"\1\2", t)
        t = t.replace("\r", "")
        t = re.sub(r"[ \t]{2,}", " ", t)
        # Navigations-/Müllzeilen optional herausfiltern
        t = re.sub(r"^\s*(Go Back|ON THIS PAGE|Author.*|Last revision.*|Help|Arduino\s*Docs)\s*$",
                   "", t, flags=re.I|re.M)
        t = re.sub(r"\n{2,}", "\n", t)
        return t.strip()

    cleaned = clean_html_extracted_pdf_text(raw_text)

    # 2.2 Sections erkennen
    sections = split_into_sections(cleaned)

    # 2.3 pro Section chunken
    records = []
    total_chunks = 0
    for s_idx, (section_title, section_text) in enumerate(sections, start=1):
        if not section_text.strip():
            continue
        raw_chunks = chunk_section(section_text, target_tokens, overlap_tokens)
        for c_idx, ch in enumerate(raw_chunks, start=1):
            total_chunks += 1
            chunk_id = f"{category}::{product}::{base_id}::s{s_idx}::c{c_idx}"
            rec = {
                "category": category,
                "chunk_id": chunk_id,
                "chunk_size": int(len(ch)),                # oder approx_token_count(ch)
                "chunk_type": "semantic",                  # frei setzbar: "semantic"
                "product": product,
                "section": section_title or "",
                "semantic_density": float(semantic_density_heuristic(ch)),
                "text": ch,
                "total_chunks": 0,                         # füllen wir nachher
            }
            records.append(rec)

    # 2.4 total_chunks nachtragen
    for rec in records:
        rec["total_chunks"] = total_chunks

    # 2.5 JSONL speichern
    out_path = Path(out_dir) / category / product
    out_path.mkdir(parents=True, exist_ok=True)
    out_file = out_path / "chunks_sample.jsonl"
    with open(out_file, "w", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    return str(out_file), total_chunks

# ======= Beispiel-Aufruf =======
# raw_text = ...   # kommt von deiner PaddleOCR-Pipeline
category = "Arduino Cloud"
product  = "Cloud Editor"
out_dir  = "main/out"
out_file, n = chunk_ocr_text_to_jsonl(raw_text, category, product, out_dir)
print("Geschrieben:", out_file, " | Chunks:", n)


In [0]:
df = spark.read.json("file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/main/out/Arduino Cloud/Cloud Editor/chunks_sample.jsonl")
df.display()

In [0]:
from langchain_text_splitters import HTMLHeaderTextSplitter

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

In [0]:
!pip install betterhtmlchunking

In [0]:

from betterhtmlchunking import DomRepresentation
from betterhtmlchunking.main import ReprLengthComparisionBy
from betterhtmlchunking.main import tag_list_to_filter_out

html_content = """
<html>
  <body>
    <div id="content">
      <h1>Document Title</h1>
      <p>First paragraph...</p>
      <p>Second paragraph...</p>
    </div>
  </body>
</html>
"""

# Create document representation with 20 character chunks.
dom_repr = DomRepresentation(
    MAX_NODE_REPR_LENGTH=20,
    website_code=html_content,
    repr_length_compared_by=ReprLengthComparisionBy.HTML_LENGTH
    # tag_list_to_filter_out=["/head", "/header", "..."]  # By default tag_list_to_filter_out is used.
)
dom_repr.start()

# Render HTML:
for idx in dom_repr.tree_regions_system.sorted_roi_by_pos_xpath:
    print("*" * 50)
    print(f"IDX: {idx}")
    roi_html_render: str =\
        dom_repr.render_system.get_roi_html_render_with_pos_xpath(
            roi_idx=idx
        )
    print(roi_html_render)


# Render text:
for idx in dom_repr.tree_regions_system.sorted_roi_by_pos_xpath:
    print("*" * 50)
    print(f"IDX: {idx}")
    roi_text_render: str =\
        dom_repr.render_system.get_roi_text_render_with_pos_xpath(
            roi_idx=idx
        )
    print(roi_text_render)


In [0]:
import re
from collections import Counter

# ---------- kleine Helfer ----------
URL_RE   = re.compile(r'https?://\S+|www\.\S+', re.I)
TAG_RE   = re.compile(r'<[^>]+>')                    # übrig gebliebene HTML-Tags
SPACES   = re.compile(r'[ \t]{2,}')
MULTI_NL = re.compile(r'\n{3,}')
PAGE_NUM_LINE = re.compile(r'^\s*\d+\s*/\s*\d+\s*$') # "12 / 26" etc.

def _strip_urls_keep_anchor(text: str) -> str:
    # [label](url)  -> label
    text = re.sub(r'\[([^\]]+)\]\((?:https?://|www\.)[^\)]+\)', r'\1', text)
    # "label (https://...)" -> "label"
    text = re.sub(r'\s*\((?:https?://|www\.)[^\)]+\)', '', text)
    # nackte URLs löschen
    text = URL_RE.sub('', text)
    return text

def _join_wrapped_lines(s: str) -> str:
    # join: Zeilen, die künstlich umbrechen (HTML->PDF)
    out = []
    prev = ''
    for ln in s.split('\n'):
        l = ln.strip()
        if not prev:
            prev = l
            continue
        # join, wenn vorige Zeile nicht mit Satzende endet UND diese klein beginnt
        if prev and l and not re.search(r'[.;:!?…]$', prev) and l[:1].islower():
            prev = prev + ' ' + l
        else:
            out.append(prev)
            prev = l
    if prev:
        out.append(prev)
    return '\n'.join(out)

def _drop_frequent_header_footer(text: str, freq_threshold=0.6):
    """
    Entfernt Zeilen, die auf sehr vielen Seiten wiederkehren (Header/Footer).
    Erwartet, dass Seitenumbrüche als '\f' oder viele Leerzeilen vorkommen.
    Fallback: Wir betrachten alle Zeilen global.
    """
    lines = [ln.strip() for ln in text.split('\n') if ln.strip()]
    if not lines:
        return text
    counts = Counter(lines)
    n = len(lines)
    to_drop = {ln for ln, c in counts.items() if c / n > freq_threshold}

    cleaned = []
    for ln in text.split('\n'):
        s = ln.strip()
        if s in to_drop:
            continue
        if PAGE_NUM_LINE.match(s):
            continue
        cleaned.append(ln)
    return '\n'.join(cleaned)

def normalize_heading(title: str) -> str:
    if not title:
        return ""
    s = title.strip().lower()
    s = re.sub(r'^\s*(section|chapter|kapitel|abschnitt)\s*\d+[:.)-]*\s*', '', s, flags=re.I)
    s = re.sub(r'^\s*\d+(?:\.\d+)*\s*[:.)-]*\s*', '', s)
    s = s.rstrip(' :.-')
    s = re.sub(r'\s+', ' ', s)
    return s

HEADINGS_BLACKLIST_EQ = {
    "contents","table of contents","toc","index","references","reference documentation",
    "company information","company info","revision history","document history","legal notice",
    "trademarks","acknowledgements","glossary","contacts","contact",
    "inhalt","inhaltsverzeichnis","verzeichnis","referenzen","referenzdokumentation",
    "unternehmensinformationen","revision","versionsverlauf","rechtliche hinweise",
    "marken","danksagungen","glossar","kontakt",
}
HEADINGS_BLACKLIST_CONTAINS = {
    "reference documentation","referenzdokumentation","table of contents","inhaltsverzeichnis",
    "revision history","document history","company information","on this page"
}

def title_matches_blacklist(title: str) -> bool:
    tnorm = normalize_heading(title)
    return (tnorm in HEADINGS_BLACKLIST_EQ) or any(k in tnorm for k in HEADINGS_BLACKLIST_CONTAINS)

def _drop_sidebar_blocks(text: str) -> str:
    """
    Entfernt typische Sidebar-/TOC-Blöcke aus HTML-Seiten-PDFs, z.B.:
    - Breadcrumbs "Home / ... "
    - "ON THIS PAGE" Liste
    - rechte Navigationsspalte mit vielen Stichpunkten
    """
    # Breadcrumb-Zeilen
    text = re.sub(r'^\s*Home\s*/.*$', '', text, flags=re.I|re.M)

    # "ON THIS PAGE" bis zur nächsten Leerzeilen-Lücke oder Abschnittsüberschrift
    text = re.sub(
        r'(?ims)^\s*(on\s+this\s+page)\s*\n(?:.+\n){1,30}?(?=\n\s*\n|^[A-Z].{0,80}$|^#|\Z)',
        '',
        text
    )

    # Link-Listen/Tabellen (viele Pipes oder viele "•" in kurzer Folge)
    lines


In [0]:
out_path_docling

In [0]:
out_path_langchain = out_dir / product_category / product_name / "langchain_chunks.jsonl"
print("⬆️ out_path_langchain:",out_path_langchain)

code_path_langchain = parent_path / "langchain_chunking.py"
print("⬆️code_path_langchain:",code_path_langchain)

code_path_docling= parent_path / "docling_chunking.py"
print("⬆️code_path_langchain:",code_path_docling)



In [0]:
from pathlib import Path
import json

def process_pdf(pdf_path: Path, out_dir: Path):
    category = pdf_path.parent.parent.name
    product  = pdf_path.parent.name

    out_path = out_dir / category / "docling_chunks.jsonl"
    out_path.parent.mkdir(parents=True, exist_ok=True)

    #doc = converter.convert(str(pdf_path)).document
    raw_chunks = list(chunker.chunk(dl_doc=doc_2))
    total_chunks = len(raw_chunks)

    records = []
    for i, ch in enumerate(raw_chunks):
        text_raw = clean_text(ch.text or "")
        if len(text_raw) < 30:
            continue

        context = clean_text(chunker.contextualize(chunk=ch))
        if len(context.split()) < 25:
            continue

        if should_drop_chunk(ch, context):  # optional
            continue

        # section (defensiv)
        section = None
        hp = getattr(ch, "hierarchy_path", None)
        if isinstance(hp, list) and hp:
            last = hp[-1]
            if isinstance(last, dict):
                section = last.get("title")

        n_tokens = tokenizer.count_tokens(context)
        semantic_density = round(n_tokens / max(1, len(context)), 4)

        rec = {
            "category": category,
            "chunk_id": f"{pdf_path.stem}::c{i}",
            "chunk_size": n_tokens,
            "chunk_type": "contextualized",
            "product": product,
            "section": section,
            "semantic_density": semantic_density,
            "text": f"[Product: {product}] [Category: {category}]\n\n{context}",
            "total_chunks": total_chunks,
        }
        records.append(rec)

    with open(out_path, "a", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    print(f"[OK] {len(records)} Chunks hinzugefügt zu: {out_path}")

def iterate_product_docs(doc_root: Path, out_dir: Path):
    for pdf_path in doc_root.rglob("*.pdf"):
        # erwartet Struktur <root>/<category>/<product>/<file.pdf>
        if len(pdf_path.parts) >= 3:
            print(f"📂 {pdf_path.parent.parent.name} / {pdf_path.parent.name}")
            process_pdf(pdf_path, out_dir)



In [0]:
from pathlib import Path
import json




doc_root = parent_path / "documents"
out_dir = parent_path / "out"
# product_name = "Nano 33 BLE"
# product_category = "Nano Family"
# file_name  = "Nano_33_BLE_datasheet.pdf"

print("⬆️ doc_root:",doc_root)

print("⬆️ out_dir:",out_dir)

pdf_path = doc_root / product_category / product_name / "Nano_33_BLE_datasheet.pdf"
print("⬆️ pdf pfad:",pdf_path)

out_path_docling = out_dir / product_category / product_name / "docling_chunks.jsonl"
print("⬆️ oout_path_docling:",out_path_docling)

iterate_product_docs(doc_root, out_dir)


In [0]:
out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/out/Nano Family/docling_chunks.jsonl"
df = spark.read.json(out_path_docling)
display(df)

In [0]:
out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/out/UNO Family/docling_chunks.jsonl"
df = spark.read.json(out_path_docling)
display(df)

In [0]:
out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/out/Education/docling_chunks.jsonl"
df = spark.read.json(out_path_docling)
display(df)

In [0]:
out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/out/MKR Family/docling_chunks.jsonl"
df = spark.read.json(out_path_docling)
display(df)





In [0]:
from pyspark.sql.functions import col
df.filter(col("product")=="MKR WAN 1310").display()

In [0]:
out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out/out/MKR Family/docling_chunks.jsonl"
df_2 = spark.read.json(out_path_docling)
display(df_2)

In [0]:
df_2.filter(col("product")=="MKR WAN 1310").display()

In [0]:

(df_2.filter((col("text").contains("Arduino Cloud")) & (col("product") == "MKR WAN 1310"))).display()

In [0]:

(df.filter((col("text").contains("Arduino Cloud")) & (col("product") == "MKR WAN 1310"))).display()

In [0]:


out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/out/Portenta Family/docling_chunks.jsonl"
df = spark.read.json(out_path_docling)
display(df)

In [0]:


out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/(Clone) out/Portenta Family/docling_chunks.jsonl"
df_2 = spark.read.json(out_path_docling)
display(df_2)

In [0]:


df_2.filter(col("product")=="Portenta H7").display()

In [0]:
df.filter(col("product")=="Portenta H7").display()

In [0]:

(df.filter((col("text").contains("Application Examples")) & (col("product") == "Portenta Vision Shield"))).display()

In [0]:

(df_2.filter((col("text").contains("Application Examples")) & (col("product") == "Portenta Vision Shield"))).display()

In [0]:

(df.filter((col("text").contains("Application Examples")) )).display()

In [0]:




out_path_docling = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/out/Arduino Cloud/docling_chunks.jsonl"
df_3 = spark.read.json(out_path_docling)
display(df_3)